# **NER_by_CRF**

## **Author:** Gema De Vargas Romero

## **Master Thesis:** "Development of a Named Entity Recognition System to automatically assign tumor morphology entity mentions to health-related documents in Spanish." 

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

path='drive/My Drive/Ejemplos NER - TFM/'
!ls 'drive/My Drive/Ejemplos NER - TFM/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
 bert
 check_results2.ipynb
 check_results.ipynb
 data
 dev_set
 dev_set2
'Dictionary based NER (spacy).ipynb'
'Ehealth_Dictionary based NER (spacy).ipynb'
 last_step_cantemist.ipynb
 last_step_cantemist_TEST.ipynb
 NER_by_BERT_Cantemist_BIOESV.ipynb
 NER_by_BERT_Cantemist_Competicion.ipynb
 NER_by_BERT_Cantemist.ipynb
 NER_by_BI_LSTM_CRF_Cantemist_BIOESV_2.ipynb
 NER_by_BI_LSTM_CRF_Cantemist_BIOESV.ipynb
 NER_by_BI_LSTM_CRF_Cantemist_Competicion.ipynb
 NER_b

## **Loading libraries**

First, we must load the libraries that we will use in this notebook:

In [ ]:
!pip install sklearn-crfsuite
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_precision_score
from sklearn_crfsuite.metrics import flat_recall_score
from sklearn_crfsuite.metrics import flat_classification_report
import numpy as np
import pandas as pd
import time
import random
import matplotlib.pyplot as plt

     |████████████████████████████████| 747kB 2.6MB/s 


In [ ]:
# Library spacy
!pip install -U spacy 
#!python -m spacy validate
!python -m spacy download es_core_news_lg

import spacy

# nlp = spacy.load("es") # no longer works with updated version of spacy 2.3.1
import es_core_news_lg
nlp = es_core_news_lg.load()

     |████████████████████████████████| 10.0MB 2.7MB/s 
     |████████████████████████████████| 2.1MB 45.7MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 573.1MB 1.1MB/s 
  Created wheel for es-core-news-lg: filename=es_core_news_lg-2.3.1-cp36-none-any.whl size=573139081 sha256=90577aabb67f6cc06f7c5160642e00913b1bb2453b11f4cb4912e556643bed92
  Stored in directory: /tmp/pip-ephem-wheel-cache-lzrypxne/wheels/48/59/33/558e7f48e924c6cac0cbd3679ee7c84f5ae02964c335232e5a
Successfully built es-core-news-lg
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_lg')


In [ ]:
!pip install unidecode
import unidecode

     |████████████████████████████████| 245kB 2.8MB/s 


## **Read the files**

In [ ]:
!ls 'drive/My Drive/Ejemplos NER - TFM/data'
import pickle as pkl

df_data_complete.csv	files_txt_dev	     sentences_dev_by_cc2
df_data_dev2_2.csv	files_txt_test	     sentences_test
df_data_dev2.csv	files_txt_test_true  sentences_test_by_cc
df_data_test.csv	sentences_dev	     sentences_test_true
df_data_test_true2.csv	sentences_dev2	     sentences_test_true_by_cc
df_data_train2.csv	sentences_dev_by_cc  sentences_train


In [ ]:
# Train dataset
with open(path+'data/sentences_train', 'rb') as file: 
  sentences_train = pkl.load(file)
file.close()

df_data_train2 = pd.read_csv(path+'data/df_data_train2.csv')
print(sentences_train[0])

[('ANAMNESIS', 'NOUN', 'O', 0), ('\n', 'SPACE', 'O', 9), ('Mujer', 'NOUN', 'O', 10), ('de', 'ADP', 'O', 16), ('67', 'NUM', 'O', 19), ('años', 'NOUN', 'O', 22), ('con', 'ADP', 'O', 27), ('antecedentes', 'NOUN', 'O', 31), ('personales', 'ADJ', 'O', 44), ('de', 'ADP', 'O', 55), ('hipotiroidismo', 'NOUN', 'O', 58), ('en', 'ADP', 'O', 73), ('tratamiento', 'NOUN', 'O', 76), ('con', 'ADP', 'O', 88), ('levotiroxina', 'NOUN', 'O', 92), ('y', 'CCONJ', 'O', 105), ('fumadora', 'ADJ', 'O', 107), ('activa', 'ADJ', 'O', 116), ('de', 'ADP', 'O', 123), ('12.5', 'NUM', 'O', 126), ('paquetes', 'NOUN', 'O', 131), ('/', 'PUNCT', 'O', 139), ('año', 'NOUN', 'O', 140), ('.', 'PUNCT', 'O', 143)]


In [ ]:
# Development dataset 1
with open(path+'data/sentences_dev', 'rb') as file: 
  sentences_dev = pkl.load(file)
file.close()

with open(path+'data/sentences_dev_by_cc', 'rb') as file: 
  sentences_dev_by_cc = pkl.load(file)
file.close()

df_data_dev2 = pd.read_csv(path+'data/df_data_dev2.csv')

print(sentences_dev_by_cc[0][0])

[('Anamnesis', 'NOUN', 'O', 0), ('\n', 'SPACE', 'O', 9), ('Varón', 'PROPN', 'O', 10), ('de', 'ADP', 'O', 16), ('74', 'NUM', 'O', 19), ('años', 'NOUN', 'O', 22), (',', 'PUNCT', 'O', 26), ('exfumador', 'NOUN', 'O', 28), ('desde', 'ADP', 'O', 38), ('hace', 'AUX', 'O', 44), ('15', 'NUM', 'O', 49), ('años', 'NOUN', 'O', 52), (',', 'PUNCT', 'O', 56), ('con', 'ADP', 'O', 58), ('único', 'ADJ', 'O', 62), ('antecedente', 'NOUN', 'O', 68), ('de', 'ADP', 'O', 80), ('hipertensión', 'NOUN', 'O', 83), (',', 'PUNCT', 'O', 95), ('dislipemia', 'NOUN', 'O', 97), ('y', 'CCONJ', 'O', 108), ('apendicectomizado', 'ADJ', 'O', 110), (';', 'PUNCT', 'O', 127), ('se', 'PRON', 'O', 129), ('diagnostica', 'VERB', 'O', 132), ('en', 'ADP', 'O', 144), ('marzo', 'INTJ', 'O', 147), ('de', 'ADP', 'O', 153), ('2013', 'NUM', 'O', 156), ('de', 'ADP', 'O', 161), ('carcinoma', 'INTJ', 'B-MOR', 164), ('de', 'ADP', 'I-MOR', 174), ('células', 'NOUN', 'I-MOR', 177), ('transicionales', 'ADJ', 'I-MOR', 185), ('de', 'ADP', 'I-MOR', 2

In [ ]:
# Development dataset 2
with open(path+'data/sentences_dev2', 'rb') as file: 
  sentences_dev2 = pkl.load(file)
file.close()

with open(path+'data/sentences_dev_by_cc2', 'rb') as file: 
  sentences_dev_by_cc2 = pkl.load(file)
file.close()

df_data_dev2_2 = pd.read_csv(path+'data/df_data_dev2_2.csv')

print(sentences_dev_by_cc2[0][0])

[('Anamnesis', 'NOUN', 'O', 0), ('\n', 'SPACE', 'O', 9), ('Mujer', 'NOUN', 'O', 10), ('de', 'ADP', 'O', 16), ('60', 'NUM', 'O', 19), ('años', 'NOUN', 'O', 22), ('diagnosticada', 'ADJ', 'O', 27), ('de', 'ADP', 'O', 41), ('enfermedad', 'NOUN', 'O', 44), ('pulmonar', 'ADJ', 'O', 55), ('obstructiva', 'ADJ', 'O', 64), ('crónica', 'ADJ', 'O', 76), ('y', 'CCONJ', 'O', 84), ('trastorno', 'NOUN', 'O', 86), ('bipolar', 'ADJ', 'O', 96), ('de', 'ADP', 'O', 104), ('años', 'NOUN', 'O', 107), ('de', 'ADP', 'O', 112), ('evolución', 'NOUN', 'O', 115), (',', 'PUNCT', 'O', 124), ('en', 'ADP', 'O', 126), ('tratamiento', 'NOUN', 'O', 129), ('con', 'ADP', 'O', 141), ('carbamazepina', 'NOUN', 'O', 145), ('hasta', 'ADP', 'O', 159), ('la', 'DET', 'O', 165), ('fecha', 'NOUN', 'O', 168), (',', 'PUNCT', 'O', 173), ('sin', 'ADP', 'O', 175), ('otros', 'DET', 'O', 179), ('antecedentes', 'NOUN', 'O', 185), ('personales', 'ADJ', 'O', 198), ('de', 'ADP', 'O', 209), ('interés', 'NOUN', 'O', 212), ('.', 'PUNCT', 'O', 219

In [ ]:
# TEST AND BACKGROUND FILES:
with open(path+'data/sentences_test', 'rb') as file: 
  sentences_test = pkl.load(file)
file.close()

with open(path+'data/sentences_test_by_cc', 'rb') as file: 
  sentences_test_by_cc = pkl.load(file)
file.close()

df_data_test = pd.read_csv(path+'data/df_data_test.csv')

print(sentences_test_by_cc[0][0])

[('Paciente', 'PROPN', 0), ('mujer', 'NOUN', 9), (',', 'PUNCT', 14), ('75', 'NUM', 16), ('años', 'NOUN', 19), ('consulta', 'VERB', 24), ('el', 'DET', 33), ('4-6-2003', 'NOUN', 36), (',', 'PUNCT', 44), ('refiriendo', 'VERB', 46), ('como', 'SCONJ', 57), ('antecedentes', 'NOUN', 62), ('personales', 'ADJ', 75), (':', 'PUNCT', 85), ('Alergia', 'PROPN', 87), ('a', 'ADP', 95), ('salicilatos', 'NOUN', 97), ('.', 'PUNCT', 108)]


## **Model 1: NER_by_CRF code:**

In [ ]:
sentences_train_COPY = sentences_train.copy()

sentences_dev_COPY = sentences_dev.copy()
sentences_dev_by_cc_COPY = sentences_dev_by_cc.copy()

sentences_dev2_COPY = sentences_dev2.copy()
sentences_dev_by_cc2_COPY = sentences_dev_by_cc2.copy()

sentences_test_COPY = sentences_test.copy()
sentences_test_by_cc_COPY = sentences_test_by_cc.copy()

In [ ]:
def preprocess_word(w):
  # 1st turn to lowercase
  w = w.lower()
  # 2nd remove accents and strange characters
  w = unidecode.unidecode(w)
  return w

In [ ]:
preprocess = True

if (preprocess==True):
  sentences_train = [[(preprocess_word(w[0]),w[1],w[2],w[3]) for w in s] for s in sentences_train]

  sentences_dev = [[(preprocess_word(w[0]),w[1],w[2],w[3]) for w in s] for s in sentences_dev]
  sentences_dev_by_cc = [[[(preprocess_word(w[0]),w[1],w[2],w[3]) for w in s] for s in cc] for cc in sentences_dev_by_cc]

  sentences_dev2 = [[(preprocess_word(w[0]),w[1],w[2],w[3]) for w in s] for s in sentences_dev2]
  sentences_dev_by_cc2 = [[[(preprocess_word(w[0]),w[1],w[2],w[3]) for w in s] for s in cc] for cc in sentences_dev_by_cc2]

  sentences_test = [[(preprocess_word(w[0]),w[1],w[2]) for w in s] for s in sentences_test]
  sentences_test_by_cc = [[[(preprocess_word(w[0]),w[1],w[2]) for w in s] for s in cc] for cc in sentences_test_by_cc]


else:
  sentences_train = sentences_train_COPY

  sentences_dev = sentences_dev_COPY
  sentences_dev_by_cc = sentences_dev_by_cc_COPY

  sentences_dev2 = sentences_dev2_COPY
  sentences_dev_by_cc2 = sentences_dev_by_cc2_COPY

  sentences_test = sentences_test_COPY
  sentences_test_by_cc = sentences_test_by_cc_COPY

In [ ]:
print(sentences_test_by_cc[0][0])

[('anamnesis', 'NOUN', 'O', 0), ('\n', 'SPACE', 'O', 9), ('se', 'PRON', 'O', 10), ('trata', 'VERB', 'O', 13), ('de', 'ADP', 'O', 19), ('un', 'DET', 'O', 22), ('paciente', 'NOUN', 'O', 25), ('de', 'ADP', 'O', 34), ('67', 'NUM', 'O', 37), ('anos', 'NOUN', 'O', 40), ('en', 'ADP', 'O', 45), ('el', 'DET', 'O', 48), ('momento', 'NOUN', 'O', 51), ('del', 'ADP', 'O', 59), ('diagnostico', 'NOUN', 'O', 63), (',', 'PUNCT', 'O', 74), ('con', 'ADP', 'O', 76), ('antecedentes', 'NOUN', 'O', 80), ('personales', 'ADJ', 'O', 93), ('a', 'ADP', 'O', 104), ('destacar', 'VERB', 'O', 106), ('de', 'ADP', 'O', 115), ('hipercolesterolemia', 'NOUN', 'O', 118), ('en', 'ADP', 'O', 138), ('tratamiento', 'NOUN', 'O', 141), ('con', 'ADP', 'O', 153), ('estatina', 'NOUN', 'O', 157), ('y', 'CCONJ', 'O', 166), ('alteracion', 'NOUN', 'O', 168), ('basal', 'ADJ', 'O', 179), ('de', 'ADP', 'O', 185), ('la', 'DET', 'O', 188), ('glucemia', 'NOUN', 'O', 191), ('en', 'ADP', 'O', 200), ('controles', 'NOUN', 'O', 203), (',', 'PUNCT

In [ ]:
from sklearn_crfsuite import CRF

### **Feature Preparation**




In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:], # last 3 characters of the word
        'word[-2:]': word[-2:], # last 2 characters of the word
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True # Beggining of sentence

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True # End of sentence

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label, start_pos in sent]

def sent2tokens(sent):
    return [token for token, postag, label, start_pos in sent]

def sent2start_pos(sent):
    return [start_pos for token, postag, label, start_pos in sent]

def sent2tokens_test(sent):
    return [token for token, postag, start_pos in sent]

def sent2start_pos_test(sent):
    return [start_pos for token, postag, start_pos in sent]

#### **Train and development datasets**

In [ ]:
# TRAIN SET
X_train = [sent2features(s) for s in sentences_train] # function called over every sentence in sentences
y_train = [sent2labels(s) for s in sentences_train]

# --------------------------------------

# DEVELOPMENT SET 1 (done by clinical case)
X_dev_cc = [[sent2features(s) for s in sentences_cc] for sentences_cc in sentences_dev_by_cc]
y_dev_cc = [[sent2labels(s) for s in sentences_cc] for sentences_cc in sentences_dev_by_cc]

start_char_dev_cc = [[sent2start_pos(s) for s in sentences_cc] for sentences_cc in sentences_dev_by_cc]
token_dev_cc = [[sent2tokens(s) for s in sentences_cc] for sentences_cc in sentences_dev_by_cc]


# DEVELOPMENT SET 1 (without considering clinical cases independently)
X_dev = [sent2features(s) for s in sentences_dev]
y_dev = [sent2labels(s) for s  in sentences_dev]

# --------------------------------------

# DEVELOPMENT SET 2 (done by clinical case)
X_dev_cc2 = [[sent2features(s) for s in sentences_cc] for sentences_cc in sentences_dev_by_cc2]
y_dev_cc2 = [[sent2labels(s) for s in sentences_cc] for sentences_cc in sentences_dev_by_cc2]

start_char_dev_cc2 = [[sent2start_pos(s) for s in sentences_cc] for sentences_cc in sentences_dev_by_cc2]
token_dev_cc2 = [[sent2tokens(s) for s in sentences_cc] for sentences_cc in sentences_dev_by_cc2]

# DEVELOPMENT SET 2 (without considering clinical cases independently)
X_dev2 = [sent2features(s) for s in sentences_dev2]
y_dev2 = [sent2labels(s) for s  in sentences_dev2]


# --------------------------------------

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)


In [ ]:
# checking
print("Number of words in the first sentence of the first clinical case: %d" %len(sentences_dev_by_cc[0][0]))
x_aux = sent2features(sentences_dev_by_cc[0][0])

# it creates a feature for every word in the sentence

print("Number of features created from the first sentence: %d" %len(x_aux))
print("\nFirst feature in first sentence:")
print(x_aux[0]) # Feature information of the next word at the beggining +1:postag. Ex. first word is 'En' BUT aux[0] contains info of 'la'

y_aux = sent2labels(sentences_dev_by_cc[0][0])
print("\nNumber of labels created from the first sentence: %d" %len(y_aux))
print("\nLabels in first sentence:")
print(y_aux)

Number of words in the first sentence of the first clinical case: 43
Number of features created from the first sentence: 43

First feature in first sentence:
{'bias': 1.0, 'word.lower()': 'anamnesis', 'word[-3:]': 'sis', 'word[-2:]': 'is', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'NOUN', 'postag[:2]': 'NO', 'BOS': True, '+1:word.lower()': '\n', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'SPACE', '+1:postag[:2]': 'SP'}

Number of labels created from the first sentence: 43

Labels in first sentence:
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'E-MOR', 'O', 'O', 'O', 'O', 'O']


#### **Test and background dataset**

In [ ]:
# TEST SET (done by clinical case)
X_test_cc = [[sent2features(s) for s in sentences_cc] for sentences_cc in sentences_test_by_cc]
start_char_test_cc = [[sent2start_pos_test(s) for s in sentences_cc] for sentences_cc in sentences_test_by_cc]
token_test_cc = [[sent2tokens_test(s) for s in sentences_cc] for sentences_cc in sentences_test_by_cc]

In [ ]:
print(sentences_test_by_cc[1][1])
print(start_char_test_cc[1][1])
print(token_test_cc[1][1])

[('en', 'ADP', 515), ('los', 'DET', 518), ('analisis', 'NOUN', 522), (',', 'PUNCT', 530), ('presentaba', 'AUX', 532), ('ligero', 'ADJ', 543), ('aumento', 'NOUN', 550), ('de', 'ADP', 558), ('la', 'DET', 561), ('cortisoluria', 'PROPN', 564), ('(', 'PUNCT', 577), ('284.5', 'SYM', 578), ('mcgr./24h', 'PROPN', 584), ('.', 'PUNCT', 593)]
[515, 518, 522, 530, 532, 543, 550, 558, 561, 564, 577, 578, 584, 593]
['en', 'los', 'analisis', ',', 'presentaba', 'ligero', 'aumento', 'de', 'la', 'cortisoluria', '(', '284.5', 'mcgr./24h', '.']


### **Train and evaluate the CRF model**



In [ ]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)

crf.fit(X_train, y_train)

# labels used in the classification report
labels = ['B-MOR', 'I-MOR', 'E-MOR', 'S-MOR', 'V-MOR']

##### **Predictions over train and development sets**

**Predictions over validation set**

In [ ]:
#Predicting on the test set.
y_pred = crf.predict(X_valid)
# returns predicted labels

f1_score_crf = flat_f1_score(y_valid, y_pred, average = 'micro', labels = labels)
precision_crf = flat_precision_score(y_valid, y_pred, average = 'micro', labels = labels)
recall_crf = flat_recall_score(y_valid, y_pred, average = 'micro', labels = labels)

print(f1_score_crf)
print(precision_crf)
print(recall_crf)

report = flat_classification_report(y_valid, y_pred, labels)
print(report)


0.7656249999999999
0.8249804228660924
0.7142372881355932


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-MOR       0.87      0.73      0.79       630
       I-MOR       0.77      0.61      0.68      1032
       E-MOR       0.79      0.66      0.72       629
       S-MOR       0.88      0.91      0.90       658
       V-MOR       0.00      0.00      0.00         1

   micro avg       0.82      0.71      0.77      2950
   macro avg       0.66      0.58      0.62      2950
weighted avg       0.82      0.71      0.76      2950



**Predictions over development set 1**

The predictions over the development set are obtained by clinical case. Therefore, loop over all clinical cases.

In [ ]:
#Predicting on the development set.
y_pred_dev = crf.predict(X_dev)

f1_score_crf_dev = flat_f1_score(y_dev, y_pred_dev, average = 'micro', labels = labels)
precision_crf_dev = flat_precision_score(y_dev, y_pred_dev, average = 'micro', labels = labels)
recall_crf_dev = flat_recall_score(y_dev, y_pred_dev, average = 'micro', labels = labels)

print(f1_score_crf_dev)
print(precision_crf_dev)
print(recall_crf_dev)

report_dev = flat_classification_report(y_dev, y_pred_dev, labels)
print(report_dev)


0.7371190528392895
0.823481384715872
0.6671517396481016
              precision    recall  f1-score   support

       B-MOR       0.85      0.70      0.76      1639
       I-MOR       0.81      0.57      0.67      2579
       E-MOR       0.74      0.60      0.67      1634
       S-MOR       0.89      0.86      0.88      1689
       V-MOR       0.00      0.00      0.00        18

   micro avg       0.82      0.67      0.74      7559
   macro avg       0.66      0.55      0.59      7559
weighted avg       0.82      0.67      0.73      7559



In [ ]:
def tokens_to_words(tokens_sent,label_sent, true_label_sent, start_char_pos):
  new_tok, new_lab, true_lab, new_start_pos = [], [], [], []

  for tokens, labels, true_labels, start_chars in zip(tokens_sent, label_sent, true_label_sent, start_char_pos):
    new_tok_aux, new_lab_aux, true_lab_aux, new_start_pos_aux = [], [], [], []
    for token, label, true_label, start_char_i in zip(tokens, labels,true_labels,start_chars):
      if token != "PAD":
        new_tok_aux.append(token)
        new_lab_aux.append(label)
        true_lab_aux.append(true_label)
        new_start_pos_aux.append(start_char_i)

    new_lab.append(new_lab_aux)
    true_lab.append(true_lab_aux)
    new_tok.append(new_tok_aux)
    new_start_pos.append(new_start_pos_aux)

  return new_tok, new_lab, true_lab, new_start_pos

In [ ]:
new_tokens_cc, new_labels_cc, true_labels_cc, new_start_pos_cc = [], [], [], []
new_tokens_all_dev, new_labels_all_dev, true_labels_all_dev, new_start_pos_all_dev = [], [], [], []

label_indices_sent_cc = []
tokens_sent_cc = []
for cc in range(len(X_dev_cc)):
  y_pred_dev = crf.predict(X_dev_cc[cc])
  y_dev_true = y_dev_cc[cc]

  new_tokens, new_labels, true_labels, new_start_pos = [], [], [], []
  new_tokens, new_labels, true_labels, new_start_pos = tokens_to_words(token_dev_cc[cc], 
                                        y_pred_dev, y_dev_true, start_char_dev_cc[cc])
  new_tokens_cc.append(new_tokens)
  new_labels_cc.append(new_labels)
  true_labels_cc.append(true_labels)
  new_start_pos_cc.append(new_start_pos)

  new_tokens_all_dev.extend(new_tokens)
  new_labels_all_dev.extend(new_labels)
  true_labels_all_dev.extend(true_labels)
  new_start_pos_all_dev.extend(new_start_pos)

report_dev_glob = flat_classification_report(y_true = true_labels_all_dev, y_pred = new_labels_all_dev, labels = labels)
print(report_dev_glob)

**Predictions over development set 2**


In [ ]:
#Predicting on the development set.
y_pred_dev2 = crf.predict(X_dev2)

f1_score_crf_dev = flat_f1_score(y_dev2, y_pred_dev2, average = 'micro', labels = labels)
precision_crf_dev = flat_precision_score(y_dev2, y_pred_dev2, average = 'micro', labels = labels)
recall_crf_dev = flat_recall_score(y_dev2, y_pred_dev2, average = 'micro', labels = labels)

print(f1_score_crf_dev)
print(precision_crf_dev)
print(recall_crf_dev)

report_dev2 = flat_classification_report(y_dev2, y_pred_dev2, labels)
print(report_dev2)


0.7696570834776584
0.8181148748159057
0.7266187050359713


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-MOR       0.84      0.77      0.81      1277
       I-MOR       0.79      0.63      0.70      2185
       E-MOR       0.75      0.67      0.71      1276
       S-MOR       0.89      0.89      0.89      1375
       V-MOR       0.00      0.00      0.00         3

   micro avg       0.82      0.73      0.77      6116
   macro avg       0.65      0.59      0.62      6116
weighted avg       0.81      0.73      0.77      6116



In [ ]:
new_tokens_cc, new_labels_cc, true_labels_cc, new_start_pos_cc = [], [], [], []
new_tokens_all_dev2, new_labels_all_dev2, true_labels_all_dev2, new_start_pos_all_dev2 = [], [], [], []

label_indices_sent_cc = []
tokens_sent_cc = []
for cc in range(len(X_dev_cc2)):
  y_pred_dev = crf.predict(X_dev_cc2[cc])
  y_dev_true = y_dev_cc2[cc]

  new_tokens, new_labels, true_labels, new_start_pos = [], [], [], []
  new_tokens, new_labels, true_labels, new_start_pos = tokens_to_words(token_dev_cc[cc], 
                                        y_pred_dev, y_dev_true, start_char_dev_cc[cc])
  new_tokens_cc.append(new_tokens)
  new_labels_cc.append(new_labels)
  true_labels_cc.append(true_labels)
  new_start_pos_cc.append(new_start_pos)

  new_tokens_all_dev2.extend(new_tokens)
  new_labels_all_dev2.extend(new_labels)
  true_labels_all_dev2.extend(true_labels)
  new_start_pos_all_dev2.extend(new_start_pos)

report_dev_glob = flat_classification_report(y_true = true_labels_all_dev2, y_pred = new_labels_all_dev2, labels = labels)
print(report_dev_glob)

##### **Confusion matrix**

In [ ]:
def confusion_matrix(actual, predicted):
    #classes       = np.unique(np.concatenate((actual,predicted)))
    classes = ['B-MOR', 'I-MOR', 'E-MOR', 'S-MOR', 'V-MOR', 'O']
    confusion_mtx = np.empty((len(classes),len(classes)),dtype=np.int)
    for i,a in enumerate(classes):
        for j,p in enumerate(classes):
            value = sum([sum([np.where((actual[sent][word]==a)*(predicted[sent][word]==p))[0].shape[0] 
                              for word in range(len(actual[sent]))]) for sent in range(len(actual))])
            #confusion_mtx[i,j] = sum([np.where((actual[sent]==a)*(predicted[sent]==p))[0].shape[0] for sent in range(len(actual))])
            confusion_mtx[i,j] = value
    return confusion_mtx

In [ ]:
# code to extract METRICS FOR ENTITY AND NON-ENTITY from the confusion matrix

def metrics_from_cm(cm,labels):
  TP = [[v for v in value[0:len(labels)]] for value in cm[0:len(labels)]]
  print(TP)
  TP = np.array(TP)
  TP = sum(sum(TP))

  FN = [value[-1] for value in cm[0:len(labels)]] # last column is O
  print(FN)
  FN = np.array(FN) 
  FN = sum(FN)

  FP = cm[len(labels)][0:len(labels)] # last column is O
  print(FP)
  FP = sum(FP)

  TN = cm[len(labels)][len(labels)] # last column is O
  print(TN)

  return TP, FN, FP, TN

**Train set**

In [ ]:
# not done by clinical case
actual    = np.array(y_valid)
predicted = np.array(y_pred)
confusion_matrix_train = confusion_matrix(actual,predicted)

df = pd.DataFrame(confusion_matrix_train, columns = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'], index = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'])
df

,B-MOR,I-MOR,E-MOR,S-MOR,V-MOR,O
B-MOR,459,9,0,43,0,119
I-MOR,9,633,41,5,0,344
E-MOR,0,28,417,19,0,165
S-MOR,16,2,6,598,0,36
V-MOR,0,0,0,1,0,0
O,44,152,62,10,0,87627


In [ ]:
TP, FN, FP, TN = metrics_from_cm(confusion_matrix_train,labels)
print(TP)
print(FN)
print(FP)
print(TN)

[[459, 9, 0, 43, 0], [9, 633, 41, 5, 0], [0, 28, 417, 19, 0], [16, 2, 6, 598, 0], [0, 0, 0, 1, 0]]
[119, 344, 165, 36, 0]
[ 44 152  62  10   0]
87627
2286
664
268
87627


**Development set 1**

In [ ]:
# confusion matrix computed over all the clinical cases 

actual    = np.array(true_labels_all_dev)
predicted = np.array(new_labels_all_dev)
cm = confusion_matrix(actual,predicted)

df = pd.DataFrame(cm, columns = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'], index = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'])
df

,B-MOR,I-MOR,E-MOR,S-MOR,V-MOR,O
B-MOR,1140,24,6,97,0,372
I-MOR,24,1458,130,10,6,951
E-MOR,1,82,987,39,3,522
S-MOR,39,2,27,1458,0,163
V-MOR,4,7,4,3,0,0
O,138,229,173,33,0,217272


In [ ]:
TP, FN, FP, TN = metrics_from_cm(cm,labels)
print(TP)
print(FN)
print(FP)
print(TN)

[[1140, 24, 6, 97, 0], [24, 1458, 130, 10, 6], [1, 82, 987, 39, 3], [39, 2, 27, 1458, 0], [4, 7, 4, 3, 0]]
[372, 951, 522, 163, 0]
[138 229 173  33   0]
217272
5551
2008
573
217272


**Development set 2**

In [ ]:
# confusion matrix computed over all the clinical cases 

actual    = np.array(true_labels_all_dev2)
predicted = np.array(new_labels_all_dev2)
cm2 = confusion_matrix(actual,predicted)

df = pd.DataFrame(cm2, columns = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'], index = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'])
df

,B-MOR,I-MOR,E-MOR,S-MOR,V-MOR,O
B-MOR,983,16,4,86,0,188
I-MOR,17,1383,93,6,0,686
E-MOR,2,65,859,35,0,315
S-MOR,36,4,19,1219,0,97
V-MOR,0,2,1,0,0,0
O,127,277,170,28,0,176467


In [ ]:
TP, FN, FP, TN = metrics_from_cm(cm2,labels)
print(TP)
print(FN)
print(FP)
print(TN)

[[983, 16, 4, 86, 0], [17, 1383, 93, 6, 0], [2, 65, 859, 35, 0], [36, 4, 19, 1219, 0], [0, 2, 1, 0, 0]]
[188, 686, 315, 97, 0]
[127 277 170  28   0]
176467
4830
1286
602
176467


### **Final CRF model**

##### **Costruct complete dataset**
by merging train and development datasets

In [ ]:
print(sentences_dev[0])

[('anamnesis', 'NOUN', 'O', 0), ('\n', 'SPACE', 'O', 9), ('varon', 'PROPN', 'O', 10), ('de', 'ADP', 'O', 16), ('74', 'NUM', 'O', 19), ('anos', 'NOUN', 'O', 22), (',', 'PUNCT', 'O', 26), ('exfumador', 'NOUN', 'O', 28), ('desde', 'ADP', 'O', 38), ('hace', 'AUX', 'O', 44), ('15', 'NUM', 'O', 49), ('anos', 'NOUN', 'O', 52), (',', 'PUNCT', 'O', 56), ('con', 'ADP', 'O', 58), ('unico', 'ADJ', 'O', 62), ('antecedente', 'NOUN', 'O', 68), ('de', 'ADP', 'O', 80), ('hipertension', 'NOUN', 'O', 83), (',', 'PUNCT', 'O', 95), ('dislipemia', 'NOUN', 'O', 97), ('y', 'CCONJ', 'O', 108), ('apendicectomizado', 'ADJ', 'O', 110), (';', 'PUNCT', 'O', 127), ('se', 'PRON', 'O', 129), ('diagnostica', 'VERB', 'O', 132), ('en', 'ADP', 'O', 144), ('marzo', 'INTJ', 'O', 147), ('de', 'ADP', 'O', 153), ('2013', 'NUM', 'O', 156), ('de', 'ADP', 'O', 161), ('carcinoma', 'INTJ', 'B-MOR', 164), ('de', 'ADP', 'I-MOR', 174), ('celulas', 'NOUN', 'I-MOR', 177), ('transicionales', 'ADJ', 'I-MOR', 185), ('de', 'ADP', 'I-MOR', 2

In [ ]:
# CONSTRUCT COMPLETE DATASET

print("Number of sentences in training dataset 1: %d" %len(sentences_train))
print("Number of sentences in development dataset 1: %d" %len(sentences_dev))
print("Number of sentences in development dataset 2: %d" %len(sentences_dev2))

Number of sentences in training dataset 1: 19502
Number of sentences in development dataset 1: 9546
Number of sentences in development dataset 2: 8682


In [ ]:
sentences_complete = sentences_train.copy()
sentences_complete.extend(sentences_dev)
sentences_complete.extend(sentences_dev2)

In [ ]:
print("Number of sentences in training dataset 1: %d" %len(sentences_train))
print("Number of sentences in development dataset 1: %d" %len(sentences_dev))
print("Number of sentences in development dataset 2: %d" %len(sentences_dev2))
print("Number of sentences in combined dataset: %d" %len(sentences_complete))

Number of sentences in training dataset 1: 19502
Number of sentences in development dataset 1: 9546
Number of sentences in development dataset 2: 8682
Number of sentences in combined dataset: 37730


In [ ]:
sentences_complete[0]

[('anamnesis', 'NOUN', 'O', 0),
 ('\n', 'SPACE', 'O', 9),
 ('mujer', 'NOUN', 'O', 10),
 ('de', 'ADP', 'O', 16),
 ('67', 'NUM', 'O', 19),
 ('anos', 'NOUN', 'O', 22),
 ('con', 'ADP', 'O', 27),
 ('antecedentes', 'NOUN', 'O', 31),
 ('personales', 'ADJ', 'O', 44),
 ('de', 'ADP', 'O', 55),
 ('hipotiroidismo', 'NOUN', 'O', 58),
 ('en', 'ADP', 'O', 73),
 ('tratamiento', 'NOUN', 'O', 76),
 ('con', 'ADP', 'O', 88),
 ('levotiroxina', 'NOUN', 'O', 92),
 ('y', 'CCONJ', 'O', 105),
 ('fumadora', 'ADJ', 'O', 107),
 ('activa', 'ADJ', 'O', 116),
 ('de', 'ADP', 'O', 123),
 ('12.5', 'NUM', 'O', 126),
 ('paquetes', 'NOUN', 'O', 131),
 ('/', 'PUNCT', 'O', 139),
 ('ano', 'NOUN', 'O', 140),
 ('.', 'PUNCT', 'O', 143)]

In [ ]:
sentences_complete = [[(preprocess_word(w[0]),w[1],w[2],w[3]) for w in s] for s in sentences_complete]
sentences_test_by_cc = [[[(preprocess_word(w[0]),w[1],w[2]) for w in s] for s in cc] for cc in sentences_test_by_cc]

In [ ]:
print(sentences_test_by_cc[0][0])

[('anamnesis', 'NOUN', 'O', 0), ('\n', 'SPACE', 'O', 9), ('se', 'PRON', 'O', 10), ('trata', 'VERB', 'O', 13), ('de', 'ADP', 'O', 19), ('un', 'DET', 'O', 22), ('paciente', 'NOUN', 'O', 25), ('de', 'ADP', 'O', 34), ('67', 'NUM', 'O', 37), ('anos', 'NOUN', 'O', 40), ('en', 'ADP', 'O', 45), ('el', 'DET', 'O', 48), ('momento', 'NOUN', 'O', 51), ('del', 'ADP', 'O', 59), ('diagnostico', 'NOUN', 'O', 63), (',', 'PUNCT', 'O', 74), ('con', 'ADP', 'O', 76), ('antecedentes', 'NOUN', 'O', 80), ('personales', 'ADJ', 'O', 93), ('a', 'ADP', 'O', 104), ('destacar', 'VERB', 'O', 106), ('de', 'ADP', 'O', 115), ('hipercolesterolemia', 'NOUN', 'O', 118), ('en', 'ADP', 'O', 138), ('tratamiento', 'NOUN', 'O', 141), ('con', 'ADP', 'O', 153), ('estatina', 'NOUN', 'O', 157), ('y', 'CCONJ', 'O', 166), ('alteracion', 'NOUN', 'O', 168), ('basal', 'ADJ', 'O', 179), ('de', 'ADP', 'O', 185), ('la', 'DET', 'O', 188), ('glucemia', 'NOUN', 'O', 191), ('en', 'ADP', 'O', 200), ('controles', 'NOUN', 'O', 203), (',', 'PUNCT

In [ ]:
# TRAIN SET formed by the complete dataset
X_train = [sent2features(s) for s in sentences_complete] # function called over every sentence in sentences
y_train = [sent2labels(s) for s in sentences_complete]

In [ ]:
print(X_train[0])

[{'bias': 1.0, 'word.lower()': 'anamnesis', 'word[-3:]': 'sis', 'word[-2:]': 'is', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'NOUN', 'postag[:2]': 'NO', 'BOS': True, '+1:word.lower()': '\n', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'SPACE', '+1:postag[:2]': 'SP'}, {'bias': 1.0, 'word.lower()': '\n', 'word[-3:]': '\n', 'word[-2:]': '\n', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'SPACE', 'postag[:2]': 'SP', '-1:word.lower()': 'anamnesis', '-1:word.istitle()': False, '-1:word.isupper()': False, '-1:postag': 'NOUN', '-1:postag[:2]': 'NO', '+1:word.lower()': 'mujer', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'NOUN', '+1:postag[:2]': 'NO'}, {'bias': 1.0, 'word.lower()': 'mujer', 'word[-3:]': 'jer', 'word[-2:]': 'er', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'NOUN', 'postag[:2]': 'NO', '-1:word.lower()': '\n',

##### **Train the model**

In [ ]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)

crf.fit(X_train, y_train)

# labels used in the classification report
labels = ['B-MOR', 'I-MOR', 'E-MOR', 'S-MOR', 'V-MOR']

##### **Predictions over the test and background set**

The predictions are obtained over each clinical case independently.

In [ ]:
def tokens_to_words_test(tokens_sent,label_sent, start_char_pos):
  new_tok, new_lab, new_start_pos = [], [], []

  for tokens, labels, start_chars in zip(tokens_sent, label_sent, start_char_pos):
    new_tok_aux, new_lab_aux, new_start_pos_aux = [], [], []
    for token, label, start_char_i in zip(tokens, labels,start_chars):
      if token != "PAD":
        new_tok_aux.append(token)
        new_lab_aux.append(label)
        new_start_pos_aux.append(start_char_i)

    new_lab.append(new_lab_aux)
    new_tok.append(new_tok_aux)
    new_start_pos.append(new_start_pos_aux)

  return new_tok, new_lab, new_start_pos

In [ ]:
new_tokens_cc, new_labels_cc, new_start_pos_cc = [], [], []
new_tokens_all, new_labels_all, new_start_pos_all = [], [], []

for cc in range(len(X_test_cc)):
  y_pred_test = crf.predict(X_test_cc[cc])

  new_tokens, new_labels, new_start_pos = [], [], []
  new_tokens, new_labels, new_start_pos = tokens_to_words_test(token_test_cc[cc], 
                                        y_pred_test, start_char_test_cc[cc])
  new_tokens_cc.append(new_tokens)
  new_labels_cc.append(new_labels)
  new_start_pos_cc.append(new_start_pos)

  new_tokens_all.extend(new_tokens)
  new_labels_all.extend(new_labels)
  new_start_pos_all.extend(new_start_pos)

In [ ]:
len(new_tokens_cc)

5232

In [ ]:
import pickle as pkl

with open(path+'results_CRF/predictions/new_tokens_cc', 'wb') as file: 
  pkl.dump(new_tokens_cc, file)
file.close()

with open(path+'results_CRF/predictions/new_labels_cc', 'wb') as file: 
  pkl.dump(new_labels_cc, file)
file.close()

with open(path+'results_CRF/predictions/new_start_pos_cc', 'wb') as file: 
  pkl.dump(new_start_pos_cc, file)
file.close()